# installing libraries

In [ ]:
pip install face_recognition


: 

# defining Google api key

In [10]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/jatingrewal/Jatin/NITK/Sem 1/MIR/photo_attendance_new/photo_attendance_api_key/photo-attendance-440408-18c9a265d0b2.json"

# detects and extracts faces, then computes and saves their encodings.

In [ ]:
import os
import json
import io

: 

In [ ]:

import numpy as np
import face_recognition
from google.cloud import vision

# Initialize Google Cloud Vision client
client = vision.ImageAnnotatorClient()

output_file = '/Users/jatingrewal/Jatin/NITK/Sem 1/MIR/photo_attendance_new/student_encodings_almost_3.json'

student_photos_path = '/Users/jatingrewal/Jatin/NITK/Sem 1/MIR/photo_attendance_new/all_3'


def compute_and_save_encodings():
    student_encodings = {}

    for student_name in os.listdir(student_photos_path):
        student_folder = os.path.join(student_photos_path, student_name)
        if os.path.isdir(student_folder):
            encodings = []
            print(f"Processing {student_name}'s photos...")
            for img_file in os.listdir(student_folder):
                img_path = os.path.join(student_folder, img_file)

                # agar koi folder h to skip kardo
                if os.path.isdir(img_path):
                    print(f"Found directory inside {student_name}'s folder, skipping: {img_path}")
                    continue

                # ye bounding box dedega, google api ka use karke
                with io.open(img_path, 'rb') as image_file:
                    content = image_file.read()
                image = vision.Image(content=content)
                response = client.face_detection(image=image)

                if not response.face_annotations:
                    print(f"Can't find any face in {student_name} folder, photo: {img_file}")
                    continue

                # bounding box ki help se face ki encodings bana rha h
                bounds = response.face_annotations[0].bounding_poly.vertices
                image_np = face_recognition.load_image_file(img_path)
                top, right, bottom, left = bounds[0].y, bounds[1].x, bounds[2].y, bounds[0].x
                face_image = image_np[top:bottom, left:right]
                if face_image.size == 0:
                    print(f"Skipping {img_file} due to empty face image")
                    continue
                face_encodings = face_recognition.face_encodings(face_image)

                # Store the encoding
                if face_encodings:
                    encodings.append(face_encodings[0])

            # agar multiple images use ho rhi h, to har student ke liye mean value save karenge
            if encodings:
                student_encodings[student_name] = np.mean(encodings, axis=0).tolist()

    # encoding ki json file bana denge
    with open(output_file, 'w') as f:
        json.dump(student_encodings, f)

    print(f"Encodings saved for {len(student_encodings)} students.")

compute_and_save_encodings()

# loads saved encodings, computes the encoding for a classroom photo, and compares it with stored data.

In [ ]:
import json
import numpy as np
import face_recognition

# input file me se encodings ko load karenge
def load_student_encodings(input_file):
    with open(input_file, 'r') as f:
        return {name: np.array(encoding) for name, encoding in json.load(f).items()}

# class photo me se faces ko extract karenge or recognize karenge
def recognize_faces_in_single_photo(class_photo_path, student_encodings, threshold):

    class_photo = face_recognition.load_image_file(class_photo_path)
    face_locations = face_recognition.face_locations(class_photo)
    face_encodings = face_recognition.face_encodings(class_photo, face_locations)

    recognized_students = set()

    for encoding in face_encodings:
        best_match_name = None
        lowest_distance = float('inf')  # initial case me distance infinity lelenge

        # saari encodings ke saath match karenge, or best match lelenge
        for student_name, student_encoding in student_encodings.items():
            distance = np.linalg.norm(student_encoding - encoding)
            
            if distance < lowest_distance:
                lowest_distance = distance
                best_match_name = student_name

        # agar best match threshold se kam h to usko add kardenge
        if lowest_distance < threshold:
            recognized_students.add(best_match_name)

    return recognized_students

# ......multiple photos me jo matches aa rhe h unka union lelenge(set me daal denge basically)
def recognize_faces_in_multiple_photos(class_photo_paths, student_encodings, threshold):
    all_recognized_students = set()

    for photo_path in class_photo_paths:
        recognized_students = recognize_faces_in_single_photo(photo_path, student_encodings, threshold)
        all_recognized_students.update(recognized_students)

    return all_recognized_students

# ............... Main function jahan se encoding banane ka kaame shuru hoga
def process_classroom_photos(class_photo_paths, input_file, threshold):

    student_encodings = load_student_encodings(input_file)

    recognized_students = recognize_faces_in_multiple_photos(class_photo_paths, student_encodings, threshold)

    # Calculate absentees in the class
    all_students = set(student_encodings.keys())
    absent_students = all_students - recognized_students

    print("Recognized Students:", list(recognized_students))
    print("Absent Students:", list(absent_students))
    print("Total Students:", len(all_students))
    print("Total Recognized Students:", len(recognized_students))
    print("Total Absent Students:", len(absent_students))

    return list(recognized_students), list(absent_students)

# input_file = '/content/drive/MyDrive/photo_attendance/student_embeddings.json'
input_file = '/Users/jatingrewal/Jatin/NITK/Sem 1/MIR/photo_attendance_new/student_encodings_almost_3.json'

# Dynamically provide the list of photos
class_photo_paths = [
    # '/Users/jatingrewal/Jatin/NITK/Sem 1/MIR/photo_attendance_new/testing/class/Jatin_kratagya_manish.jpeg',
    # '/Users/jatingrewal/Jatin/NITK/Sem 1/MIR/photo_attendance_new/testing/class/km.jpeg',
    # '/Users/jatingrewal/Jatin/NITK/Sem 1/MIR/photo_attendance_new/testing/class/skm_okay.jpeg'
    # '/Users/jatingrewal/Jatin/NITK/Sem 1/MIR/photo_attendance_new/testing/almost/akshay_rahul_mayur_deobrat_2+.jpg',
    # '/Users/jatingrewal/Jatin/NITK/Sem 1/MIR/photo_attendance_new/testing/almost/right_bottom.jpg'
    # '/Users/jatingrewal/Jatin/NITK/Sem 1/MIR/photo_attendance_new/testing/almost/center.jpg'
    # '/Users/jatingrewal/Jatin/NITK/Sem 1/MIR/photo_attendance_new/testing/almost/sankethraj_noamaan_ayush_abhijeet_menovi_tughuvi.jpg',
    # '/Users/jatingrewal/Jatin/NITK/Sem 1/MIR/photo_attendance_new/testing/almost/left_bottom.jpg',
      # '/Users/jatingrewal/Jatin/NITK/Sem 1/MIR/photo_attendance_new/testing/almost/right_top.jpg',
      # '/Users/jatingrewal/Jatin/NITK/Sem 1/MIR/photo_attendance_new/testing/almost/right_top_2.jpg',
      # '/Users/jatingrewal/Jatin/NITK/Sem 1/MIR/photo_attendance_new/testing/almost/left_top_2.jpg',
    
    #   '/Users/jatingrewal/Jatin/NITK/Sem 1/MIR/photo_attendance_new/testing/almost3/left_top.jpg',
    #   '/Users/jatingrewal/Jatin/NITK/Sem 1/MIR/photo_attendance_new/testing/almost3/right_top.jpg',
    #   '/Users/jatingrewal/Jatin/NITK/Sem 1/MIR/photo_attendance_new/testing/almost3/right_bottom.jpg',
    #   '/Users/jatingrewal/Jatin/NITK/Sem 1/MIR/photo_attendance_new/testing/almost3/left_bottom.jpg',
    #   '/Users/jatingrewal/Jatin/NITK/Sem 1/MIR/photo_attendance_new/testing/almost3/left_full.jpg',
    #   '/Users/jatingrewal/Jatin/NITK/Sem 1/MIR/photo_attendance_new/testing/almost3/right_full.jpg',
    #   '/Users/jatingrewal/Jatin/NITK/Sem 1/MIR/photo_attendance_new/testing/almost3/left_bottom.jpg',
    #   '/Users/jatingrewal/Jatin/NITK/Sem 1/MIR/photo_attendance_new/testing/almost3/full class.jpg'
    
]

recognized_students, absent_students = process_classroom_photos(class_photo_paths, input_file, 0.5)

Recognized Students: ['242IT028_S.HARSHITA ', '242IT032_TARUN SAINI ', '242IT021_NANDAM SAI SAKETH', '242IT005_ABHISHEK SUNIL BHAMARE ', '242IT027_RAJESH KUMAR', '242IT017_MANISH PRAJAPATI', '242IT006_CHINMAY SATISH BHANGALE', '242IT035_YASH KACHHWAH ', '242IT036_YASHANK PATIDAR ', '242IT016_KUNDURU PHANEENDRA REDDY ', '242IT020_MITESH KUMAR MANDAL', '242IT029_SANKETHRAJ KOTAGOND ', '242IT004_ANKITA VAADIRAAJU ', '242IT011_PRAJWAL ANIL GABHANE ', '242IT015_KRATAGYA SHARMA', '242IT014_KONDURU SAI KIRAN ']
Absent Students: ['242IT026_RAHUL NEGI ', '242IT002_AKANSH GWALWANSHI ', '242IT034_AAYUSH WAGHCHAURE ', '242IT033_TUGHUVI SWU ', '242IT018_MAYUR CHHALOTRE ', '242IT031_SURAJ BHAGAT ', '242IT003_AKASH ROHIT ', '242IT009_DIVYANSH ADURI ', '242IT010_ARYA ADWAIT DONGRE ', '242IT007_DASARI CHARAN SRINIVAS KUMAR REDDY', '243IT001_ARJUN T D', '242IT023_NOAMAAN ABDUL AZEEM ', '242IT019_MENOVI YHOSHU ', '242IT022_NISHANT SAHU', '242IT030_SHELKE AKSHAY BABASAHEB ', '242IT008_DEOBRAT KUMAR JHA ',

In [18]:
pip install openpyxl

1742.56s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


In [36]:
import pandas as pd
from datetime import datetime
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
import openpyxl

def create_attendance_excel(recognized_students, absent_students, output_directory):

    current_date = datetime.now().strftime("%Y-%m-%d")
    current_time = datetime.now().strftime("%H-%M-%S")
    attendance_data = []

    for student in recognized_students:
        roll_no, name = student.split('_', 1)
        attendance_data.append([roll_no, name.strip(), 'Present'])

    for student in absent_students:
        roll_no, name = student.split('_', 1)
        attendance_data.append([roll_no, name.strip(), 'Absent'])

    attendance_df = pd.DataFrame(attendance_data, columns=['Roll No', 'Name', 'Status'])

    output_excel_file = f"{output_directory}/attendance_report_{current_date}_{current_time}.xlsx"

    workbook = Workbook()
    sheet = workbook.active

    sheet.merge_cells(start_row=1, start_column=1, end_row=2, end_column=3)
    date_cell = sheet.cell(row=1, column=1)
    date_cell.value = current_date
    date_cell.alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center')

    # Write the DataFrame to the Excel sheet starting from the second row
    for r_idx, row in enumerate(attendance_df.itertuples(index=False), start=3):
        for c_idx, value in enumerate(row, start=1):
            sheet.cell(row=r_idx, column=c_idx, value=value)

    # Optionally, adjust column widths for better readability
    for column in sheet.columns:
        max_length = 0
        column = [cell for cell in column]
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 6)
        sheet.column_dimensions[get_column_letter(column[0].column)].width = adjusted_width

    workbook.save(output_excel_file)

    print(f"Attendance data saved to {output_excel_file}")

# Define the output directory
output_directory = '/Users/jatingrewal/Jatin/NITK/Sem 1/MIR/photo_attendance_new/reports'


create_attendance_excel(recognized_students, absent_students, output_directory)

Attendance data saved to /Users/jatingrewal/Jatin/NITK/Sem 1/MIR/photo_attendance_new/reports/attendance_report_2024-11-07_12-13-29.xlsx
